In [ ]:
import os
import requests
import google.generativeai as genai

# ---------------- CONFIG ----------------

os.environ["GOOGLE_API_KEY"] = "xxx"  # PUT YOUR REAL API KEY HERE
api_key = os.environ.get("GOOGLE_API_KEY")

if not api_key:
    raise RuntimeError("Google API key is not configured")

genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-flash')

repo_url = "https://github.com/YosraGhanmi/testpythoninprompot.git"

# ---------------- GITHUB FETCH ---------------- => get the python file from the git repository

def fetch_single_python_file(repo_url: str):
    repo_url = repo_url.replace(".git", "")
    api_url = repo_url.replace(
        "https://github.com/",
        "https://api.github.com/repos/"
    ) + "/contents"

    response = requests.get(api_url)
    response.raise_for_status()

    files = response.json()
    py_files = [f for f in files if f["name"].endswith(".py")]

    if len(py_files) != 1:
        raise ValueError("Repository must contain exactly one Python file")

    file_name = py_files[0]["name"]
    raw_url = py_files[0]["download_url"]
    code = requests.get(raw_url).text

    return file_name, code

# ---------------- PROMPT ----------------

PROMPT_TEMPLATE = """
You are given a Python source file.

Your task is to generate unit tests using pytest.

Rules:
- Output ONLY valid Python code
- Use pytest
- Do NOT include explanations, comments, or markdown
- Do NOT repeat the source code
- Test normal cases, edge cases, and error cases
- Assume the module can be imported by its filename
- The output must be a complete pytest test file

Source code:
{code}
"""

# ---------------- LLM CALL ---------------- => sends request and recieve the response

def generate_tests(code: str):
    prompt = PROMPT_TEMPLATE.format(code=code)
    response = model.generate_content(prompt)
    return response.text.strip()

# ---------------- MAIN ---------------- => generates python test file

if __name__ == "__main__":
    filename, source_code = fetch_single_python_file(repo_url)
    test_code = generate_tests(source_code)

    test_filename = f"test_{filename}"
    with open(test_filename, "w", encoding="utf-8") as f:
        f.write(test_code)

    print(f"Generated tests saved to {test_filename}")


Generated tests saved to test_mycode.py
